# Exercise 1

1.  Use the [biker.png](biker.png) template to do mean shift tracking in the [traffic video](slow_traffic_small.mp4). (Hint: use OpenCV's calcBackProject() function to produce a similarity image for mean shift - [see this mean shift tutorial](https://docs.opencv.org/3.4/d7/d00/tutorial_meanshift.html) for more pointers)

Note that for mean shift tracking you need to provide an initial tracking window manually, and the biker only shows up from frame 114, so wait until then to start tracking.

In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture("exercise_materials/slow_traffic_small.mp4")
biker = cv2.imread("exercise_materials/biker.png")

ret, frame = cap.read()
count = 1

x, y, w, h = 600, 200, biker.shape[1] - 20, biker.shape[0] - 30
track_window = (x, y, w, h)

# set up the ROI for tracking
hsv_roi = cv2.cvtColor(biker, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)),
                   np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
while(count < 115):
    ret, frame = cap.read()
    count += 1

while(1):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        # Draw it on image
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), 255, 2)
        cv2.imshow('img2', img2)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break


2. What happens when the biker disappears over the horizon? Why?

MeanShift starts looking at the lines on the road, as they have a similar histogram (in terms of color) to the biker

# Exercise 2
1. Use the [biker.png](biker.png) template to do Kalman filter tracking in the [traffic video](slow_traffic_small.mp4). 

Hints: See [this Python implementation](https://raw.githubusercontent.com/tobybreckon/python-examples-cv/master/kalman_tracking_live.py) for pointers. You can define the state as the position and velocity of the biker, and use the output of mean shift or cam shift (or a detection method of your choice) for the measurement update (note: we only measure position!). You will need to define a measurement matrix, a state transition matrix (motion model), as well as covariance matrices for the measurement and process (model) noise. You can start with unit matrices, and experiment with the parameters.

2. What happens if you skip the measurement step for certain frames?

3. (EXTRA) Visualize the position uncertainty (errorCovPost attribute in OpenCV) as an ellipse. Plot the measured vs. Kalman filtered position over time and compare.